In [1]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [4]:
box_office = pd.read_csv(
    '/Users/ronlodetti/Documents/Flatiron/1_phase/Project_1/Movie_Analysis_Project/data/imported/tn.movie_budgets.csv.gz')
conn = sqlite3.connect(
    '/Users/ronlodetti/Documents/Flatiron/1_phase/Project_1/Movie_Analysis_Project/data/imported/im.db')
q = """
SELECT 
    movie_id,
    primary_title AS title,
    start_year AS year,
    runtime_minutes AS runtime
FROM movie_basics 
JOIN principals
    USING('movie_id')
JOIN persons
    USING('person_id')
WHERE job == 'producer'
GROUP BY movie_id
;
"""
movies = pd.read_sql(q, conn)

q = """
SELECT 
    movie_id,
    primary_name AS producer
FROM movie_basics 
JOIN principals
    USING('movie_id')
JOIN persons
    USING('person_id')
WHERE job == 'producer'
;
"""
producers = pd.read_sql(q, conn)

q = """
SELECT 
    movie_id,
    genres
FROM movie_basics 
JOIN principals
    USING('movie_id')
JOIN persons
    USING('person_id')
WHERE job == 'producer'
GROUP BY movie_id
;
"""
genres_df = pd.read_sql(q, conn)

In [146]:
# Box Office
def prep_box_office(df):
    # Convert currencies to integers.
    for i in ['production_budget', 'worldwide_gross']:
        df[i] = df[i].str.replace('$', '')
        df[i] = df[i].str.replace(',', '')
        df[i] = df[i].str.replace(' ', '')
        df[i] = df[i].astype(int)

    # Create a new column to calculate ROI.
    profit = df['worldwide_gross'] - df['production_budget']
    df['ROI'] = (profit/df['production_budget']) * 100

    # Extracting the year from 'release_date' column.
    df['year'] = pd.to_datetime(df['release_date']).dt.year

    # Norming and filtering column names
    df.rename(columns={'movie': 'title',
                       'production_budget': 'budget'}, inplace=True)
    for i in ['title']:
        df[i] = df[i].str.translate(str.maketrans('', '', string.punctuation))
        df[i] = df[i].str.replace(' ', '')
        df[i] = df[i].str.lower()
    df = df[['title', 'year', 'budget', 'ROI']]
    return df

In [ ]:
# Movies
def prep_movies(df):
    for i in ['title']:
        df[i] = df[i].str.translate(str.maketrans('', '', string.punctuation))
        df[i] = df[i].str.replace(' ', '')
        df[i] = df[i].str.lower()
    return df


imdb_df = imdb_df.explode('genres')

In [ ]:
# Genres
def prep_genres(df):
    df['genres'] = df['genres'].map(lambda x: x.split(','), na_action='ignore')
    return df

In [5]:
genres_df['genres'] = genres_df['genres'].map(lambda x: x.split(','), na_action='ignore')
genres_df = genres_df.explode('genres')


In [6]:
genres_df

,movie_id,genres
0,tt0069049,Drama
1,tt0100275,Comedy
1,tt0100275,Drama
1,tt0100275,Fantasy
2,tt0111414,Comedy
...,...,...
67575,tt9916160,Documentary
67576,tt9916186,Documentary
67577,tt9916538,Drama
67578,tt9916706,Comedy
